## llm 설정

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv,find_dotenv

load_dotenv()

### 1. 시스템 프롬프트 작성

In [ ]:
llm_model = ChatOllama(model='llama3.1:latest')
gpt_model = ChatOpenAI(model_name="gpt-4")

### 2. 텍스트 벡터화 임베딩

### 챗봇 ui

In [1]:
import gradio as gr


# 이벤트 처리 함수
def chatbot_send_message(chat_history, text):
    ai_text = chain.invoke(text)
    chat_history.append([text, ai_text])
    return chat_history, ""

with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"],
                       ['나는 지금 배가 고파요', '배가 고프시군요 메뉴를 추천해드릴까요?']
                       ],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=7
            ) # 채팅 입력

            gr.Audio(
                format="mp3",
                scale=1,
                min_width=200,
                label="음성을 입력해주세요"
            ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        # 텍스트 입력 파일 업로드 칸 만들고 
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_chatbot, cb_textbox], # 이걸 서버 혹은 db에서 받아오게 한다면? 협업할 때 요청 받은 값을 이 곳에 넣기
            outputs=[cb_chatbot, cb_textbox]
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass
app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7863
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
